# Notebook to run a VGG16 model on the Labelled After Fire GeoTIFF Images

#### Import necessary packages and libraries

In [28]:
import os
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
import tensorflow_addons as tfa
from sklearn.utils import shuffle


#### Define a custom image generator for GeoTIFF files using rasterio to use on the VGG16 model

In [29]:
# Define the base paths for training and testing
base_training_path =  '/Volumes/HD710PRO/Fire_and_Hurricane_Images/Fire/BinaryData/Training'
base_testing_path =  '/Volumes/HD710PRO/Fire_and_Hurricane_Images/Fire/BinaryData/Test'

# This image generator does not implement Keras preprocess_input because when using it the model was computationally expensive and it was obtaining low accuracies
def custom_image_generator(file_paths, batch_size):
    while True:
        file_paths = shuffle(file_paths)

        for i in range(0, len(file_paths), batch_size):
            batch_files = file_paths[i:i + batch_size]
            images, labels = [], []

            for file in batch_files:
                with rasterio.open(file) as src:
                    image = src.read()
                    image = np.moveaxis(image, 0, -1)  # Channels last

                label = 1 if '/Damaged/' in file else 0
                images.append(image)
                labels.append(label)

            yield np.array(images), np.array(labels)


#### Compute the file paths and split the data into training, test and validation sets

In [30]:
def get_file_paths(base_path):
    file_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.tif'):
                file_paths.append(os.path.join(root, file))
    return file_paths

train_files = get_file_paths(base_training_path)
test_files = get_file_paths(base_testing_path)

# Split training data for validation
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)


#### Define batch size and generate the training, validation and test data

In [31]:
batch_size = 32

train_generator = custom_image_generator(train_files, batch_size)
val_generator = custom_image_generator(val_files, batch_size)
test_generator = custom_image_generator(test_files, batch_size)


#### Define and compile the model

In [19]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), tfa.metrics.F1Score(num_classes=1, threshold=0.5)])


#### Fit the model

In [20]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_files) // batch_size,
    validation_data=val_generator,
    validation_steps=len(val_files) // batch_size,
    epochs=10)


Epoch 1/10
2688/2688 [==============================] - 42184s 16s/step - loss: 1.6159 - accuracy: 0.7951 - precision_2: 0.6889 - recall_2: 0.4587 - f1_score: 0.5507 - val_loss: 0.4836 - val_accuracy: 0.8216 - val_precision_2: 0.7688 - val_recall_2: 0.4973 - val_f1_score: 0.6040
Epoch 2/10
2688/2688 [==============================] - 41087s 15s/step - loss: 0.4201 - accuracy: 0.8491 - precision_2: 0.8119 - recall_2: 0.5843 - f1_score: 0.6796 - val_loss: 0.4942 - val_accuracy: 0.8318 - val_precision_2: 0.7499 - val_recall_2: 0.5754 - val_f1_score: 0.6511
Epoch 3/10
2688/2688 [==============================] - 40506s 15s/step - loss: 0.3562 - accuracy: 0.8754 - precision_2: 0.8514 - recall_2: 0.6601 - f1_score: 0.7437 - val_loss: 0.5986 - val_accuracy: 0.8380 - val_precision_2: 0.8034 - val_recall_2: 0.5404 - val_f1_score: 0.6462
Epoch 4/10
2688/2688 [==============================] - 39089s 15s/step - loss: 0.2835 - accuracy: 0.8983 - precision_2: 0.8886 - recall_2: 0.7191 - f1_score: 0

### Save and evaluate the model

In [22]:
model.save('wildfiredetectionafterVGG16.keras')


In [33]:
from tensorflow.keras.models import load_model


In [34]:
model = load_model('wildfiredetectionafterVGG16.keras')


In [35]:
test_loss, test_accuracy, test_precision, test_recall, test_f1_score = model.evaluate(
    test_generator,
    steps=len(test_files) // batch_size)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1_score}")


289/331 [=========================>....] - ETA: 14:00 - loss: 1.2443 - accuracy: 0.8317 - precision_2: 0.7516 - recall_2: 0.5889 - f1_score: 0.6604

/opt/anaconda3/lib/python3.9/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


331/331 [==============================] - 6570s 20s/step - loss: 1.2207 - accuracy: 0.8327 - precision_2: 0.7499 - recall_2: 0.5890 - f1_score: 0.6598
Test Accuracy: 0.8326565027236938
Test Precision: 0.7498908638954163
Test Recall: 0.5889612436294556
Test F1 Score: [0.65975416]
